# Notebook Vizualising Sample Results of Few Methods Avaliable in Nevergrad

In [1]:
from brian2 import *
from brian2tools import *
from nevergrad.optimization import optimizerlib
from nevergrad import instrumentation as inst

WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
 [py.warnings]


In [15]:
from nevergrad import DifferentialEvolution

ImportError: cannot import name 'DifferentialEvolution' from 'nevergrad' (/home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/nevergrad/__init__.py)

In [2]:
%matplotlib inline
prefs.codegen.target = 'cython'  # weave is not multiprocess-safe!
candidates, parameters = [], []

## Setup Model to be Fit

### Parameters

In [3]:
area = 20000*umetre**2
Cm = 1*ufarad*cm**-2 * area
El = -65*mV
EK = -90*mV
ENa = 50*mV
VT = -63*mV

### Generate a step-current input and an "experimental" voltage trace

In [4]:
dt = 0.01*ms
input_current = np.hstack([np.zeros(int(5*ms/dt)), np.ones(int(5*ms/dt)), np.zeros(int(5*ms/dt))])*nA

N, n1 = np.array([input_current]).shape
params_correct = {'gl': float(5e-5*siemens*cm**-2 * area),
          'g_na': float(100*msiemens*cm**-2 * area),
          'g_kd': float(30*msiemens*cm**-2 * area)}

defaultclock.dt = dt
I = TimedArray(input_current, dt=dt)

### The model

In [5]:
eqsHH = Equations('''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I(t))/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
g_na : siemens (constant)
g_kd : siemens (constant)
gl   : siemens (constant)
''')

G = NeuronGroup(1, eqsHH, method='exponential_euler')
G.v = El
G.set_states(params_correct, units=False)
mon = StateMonitor(G, 'v', record=0)
run(20*ms)

voltage = mon.v[0]/mV
voltage += np.random.randn(len(voltage))

inp_trace = np.array([input_current])

n0, n1 = inp_trace.shape
out_trace = np.array([voltage[:n1]])

## Setup Model for Modelfitting

### Model for modelfitting

In [6]:
eqs = Equations(
'''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
g_na : siemens (constant)
g_kd : siemens (constant)
gl   : siemens (constant)
''',
Cm = 1*ufarad*cm**-2 * area, El = -65*mV, EK = -90*mV, ENa = 50*mV, VT = -63*mV)

### setup the nevergrad optimizer

In [11]:
arg1 = inst.var.Array(1).bounded(1e-7*siemens*cm**-2 * area, 1e-3*siemens*cm**-2 * area).asscalar()
arg2 = inst.var.Array(1).bounded(1*msiemens*cm**-2 * area, 2000*msiemens*cm**-2 * area).asscalar()
arg3 = inst.var.Array(1).bounded(1*msiemens*cm**-2 * area, 1000*msiemens*cm**-2 * area).asscalar()
instrum = inst.Instrumentation(arg1, arg2, arg3)

optim = optimizerlib.registry['DE'](instrumentation=instrum, num_workers=100)

In [13]:
optim._populationsize = "large"

In [14]:
DifferentialEvolution()

NameError: name 'DifferentialEvolution' is not defined

### Get Parameters From Ask
### pass parameters to the NeuronGroup
### give information to the optimizer

In [8]:
n_rounds=2

In [9]:
for _ in range(n_rounds):
    for _ in range(20):
        cand = optim.ask()
        candidates.append(cand)
        parameters.append(list(cand.args))

    errors = fit_traces_ask_tell(model = eqs, input_var = 'I', output_var = 'v',\
            input = inp_trace * nA, output = out_trace*mV, dt = dt,
            gl = [1e-5*siemens*cm**-2 * area, 1e-6*siemens*cm**-2 * area],
            g_na = [1*msiemens*cm**-2 * area, 2000*msiemens*cm**-2 * area],
            g_kd = [1*msiemens*cm**-2 * area, 1000*msiemens*cm**-2 * area],
            update=parameters)

    for i, candidate in enumerate(candidates):
        value = errors[i]
        optim.tell(candidate, value)

    ans = optim.provide_recommendation()
    print('correct:', params_correct, '\n output:', ans.args)

NameError: name 'fit_traces_ask_tell' is not defined

### Get The Best Results

## Visualization of the results

In [ ]:
params = {'gl': ans.args[0],
          'g_na': ans.args[1],
          'g_kd': ans.args[2]}

### re-run the neuron with new params

In [ ]:
start_scope()
G = NeuronGroup(1, eqsHH, method='exponential_euler')
G.v = El
G.set_states(params, units=False)
mon = StateMonitor(G, 'v', record=0)
run(20*ms)

voltage1 = mon.v[0]/mV

In [ ]:
plt.figure()
plot(np.arange(len(voltage))*dt/ms, voltage);
plot(np.arange(len(voltage1))*dt/ms, voltage1);
plt.title('nevergrad optimization');